In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '..//')
import pandas as pd
import numpy as np

from pathlib import Path
import xarray as xr

from src.common import functions
from src.common import enums, schema

In [ ]:
target_area = enums.Area.CARDIFF
HDD_ref_temperature:float = 15.5
CDD_ref_temperature:float = 24
path_results:Path = Path().absolute().parent/"data"

input_data = functions.get_all_data(target_area.value)
input_data.tail()

In [4]:
degree_day_dataf = functions.get_degree_days_dataframe(input_data, 
                                                        HDD_ref_temperature, 
                                                        CDD_ref_temperature)
#export results
degree_day_dataf.to_csv(str(Path(path_results)/"raw"/f"{target_area.value}_degree_days.csv"))

In [5]:


#Calculate HDD and CDD based on a rolling period of 1 day, 1 week, 2 weeks

list_periods = [1, 7, 14, 21, 28] #number of days by rolling periods

for number_days in list_periods:
    target_col = f"{schema.OutputDataSchema.HDD}_{HDD_ref_temperature:0.1f}"
    temp_HDD_dataf = functions.get_rolling_average_daily_degree_days_dataf(degree_day_dataf, target_col, number_days).round(3)
    name = f"{target_area.value}_hdd_rolling_{number_days:02}_days"

    filename = f"{name}.csv"
    temp_HDD_dataf.to_csv(str(Path(path_results)/"rolling"/filename))

    target_col = f"{schema.OutputDataSchema.CDD}_{CDD_ref_temperature:0.1f}"
    temp_CDD_dataf = functions.get_rolling_average_daily_degree_days_dataf(degree_day_dataf, target_col, number_days).round(3)
    filename = f"{target_area.value}_cdd_rolling_{number_days:02}_days.csv"
    temp_CDD_dataf.to_csv(str(Path(path_results)/"rolling"/filename))
